In [ ]:
import medmnist
from medmnist import INFO, Evaluator
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import numpy as np
import sys

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
data_flag = 'chestmnist' 
info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

img_size = 28

DataClass = getattr(medmnist, info['python_class'])
train_data = DataClass(split='train', download=True, size=img_size)
test_data = DataClass(split='test', download=True, size=img_size)
val_data = DataClass(split='val', download=True, size=img_size)

X_train, y_train = train_data.imgs, train_data.labels
X_val, y_val = val_data.imgs, val_data.labels
X_test, y_test = test_data.imgs, test_data.labels

from tensorflow.keras.mixed_precision import set_global_policy

# Set the mixed precision policy to 'mixed_float16'
set_global_policy('mixed_float16')

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

In [ ]:
X_train = tf.expand_dims(X_train, axis=-1)  # Adds channel dimension at the end
X_val = tf.expand_dims(X_val, axis=-1)
X_test = tf.expand_dims(X_test, axis=-1)

In [ ]:

# Convert one-hot encoded labels to class indices
y_train = np.argmax(y_train, axis=1)
y_val = np.argmax(y_val, axis=1)
y_test = np.argmax(y_test, axis=1)

In [ ]:
# Convert all classes to 1 or 0
def convert2Binary(npArr):
    for i in range(len(npArr)):
        if(npArr[i] != 0):
            npArr[i] = 1
    return npArr
y_train = convert2Binary(y_train)
y_val = convert2Binary(y_val)
y_test = convert2Binary(y_test)

In [ ]:
from keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

def preprocess_images(images):
    images_resized = tf.image.resize(images, (32, 32))
    return images_resized

X_train_rgb = tf.image.grayscale_to_rgb(X_train)
X_test_rgb = tf.image.grayscale_to_rgb(X_test)

X_train_resized = preprocess_images(X_train_rgb)
X_test_resized = preprocess_images(X_test_rgb)



base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
output = GlobalAveragePooling2D()(base_model.output)  # Reduce to 1D embeddings
feature_extractor = Model(inputs=base_model.input, outputs=output)

In [ ]:
def extract_features(feature_extractor, images, batch_size=32):
    """
    Extract features from a dataset using the feature extractor.

    Args:
        feature_extractor: Pretrained MobileNet-based feature extractor.
        images: Dataset as a NumPy array (preprocessed and resized).
        batch_size: Number of samples per batch.

    Returns:
        Extracted features as a NumPy array.
    """
    num_samples = images.shape[0]
    features = []

    for start in range(0, num_samples, batch_size):
        end = start + batch_size
        batch = images[start:end]
        batch_features = feature_extractor.predict(batch)
        features.append(batch_features)

    return np.concatenate(features, axis=0)

train_features = extract_features(feature_extractor, X_train_resized, batch_size=32)
test_features = extract_features(feature_extractor, X_test_resized, batch_size=32)

In [ ]:
print(f"Train features shape: {train_features.shape}, Train labels shape: {y_train.shape}")
print(f"Test features shape: {test_features.shape}, Test labels shape: {y_test.shape}")


In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_val, y_train, y_val = train_test_split(train_features, y_train, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_train)
print(f"Validation Accuracy: {accuracy_score(y_train, y_pred):.4f}")

In [ ]:
X_test, X_val, y_test, y_val = train_test_split(test_features, y_test, test_size=0.2, random_state=42)
y_pred = rf_model.predict(X_test)
print(f"Validation Accuracy: {accuracy_score(y_test, y_pred):.4f}")